First, we install the required packages

In [1]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-+.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=274491 sha256=c2528781ddab4e4625bee364bffd309f75836ca58456b4c2dc85b91149770028
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-+.html
     |████████████████████████████████| 2.1 MB 4.3 MB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.15-cp37-cp37m-linux_x86_64.whl size=516860 sha256=da78f6ce81fd9611432b36d0d5319a49fddf09ed3d754f79bfeb5c5c40b41807
  Stored in directory: /root/.cache/pip/wheels/15/68/4d/1414be5c2c622bad35364e13213180797717b6d4b8923936dc
Successfully built torch-sparse
Loo

In [2]:
!pip install mendeleev

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 179 kB 4.2 MB/s 
     |████████████████████████████████| 865 kB 51.2 MB/s 
     |████████████████████████████████| 1.1 MB 46.8 MB/s 
  Attempting uninstall: Pygments
    Found existing installation: Pygments 2.6.1
    Uninstalling Pygments-2.6.1:
      Successfully uninstalled Pygments-2.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


And then, we import all required modules

In [21]:
from datetime import datetime
import pdb
import sys

import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
import yaml
import markdown
import os

from google.colab import drive
drive.mount('/content/drive')

sys.path.append('/content/drive/MyDrive/Colab Notebooks/MoleculeDB')

from callbacks import set_up_callbacks
from count_model_parameters import count_model_parameters
from features import set_up_features
from graph_potential import graph_potential
from molecule_graph_dataset import MoleculeGraphDataSet
from fit_model import fit_model
from set_up_molecular_graphs import set_up_molecular_graphs

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


A driver script to fit a Graph Convolutional Neural Network GCNN model to
represent properties of molecular/condensed matter systems.

To execute: python fit_graph.py input-file

where input-file is a yaml file specifying different parameters of the
model and how the job is to be run. For an example see sample.yml

In [22]:
# input_file is a yaml compliant file
input_file = '/content/drive/MyDrive/Colab Notebooks/MoleculeDB/sample.yml'

with open( input_file, 'r' ) as input_stream:
    input_data = yaml.load(input_stream, Loader=yaml.Loader)

Debug: we read it from yaml file; if it is set True, the debugging process begins

In [23]:
debug = input_data.get("debug", False)

if debug:
    pdb.set_trace()

We assign the loaded data

In [24]:
nGraphConvolutionLayers = input_data.get("nGraphConvolutionLayers", 1)
nFullyConnectedLayers = input_data.get("nFullyConnectedLayers", 1)
nMaxNeighbours = input_data.get("nMaxNeighbours", 6)
useCovalentRadii = input_data.get("useCovalentRadii", False)
nodeFeatures = input_data.get("nodeFeatures", ["atomic_number"])
nTotalNodeFeatures = input_data.get("nTotalNodeFeatures", 10)
nNeurons = input_data.get("nNeurons", None)

We read and set up the edge, bond-angle and dihedral-angle features we work with

In [25]:
edges, bond_angle, dihedral_angle = set_up_features(input_data)

edge_parameters = edges.parameters()
bond_angle_parameters = bond_angle.parameters()

if dihedral_angle:
    dihedral_angle_parameters = dihedral_angle.parameters()

# now the total number of edge features is given by the sum
# of edges features + 2 * bond_angle features + dihedral_angle features
# all these features define the edge entries

nTotalEdgeFeatures = edges.n_features() + 2 * bond_angle.n_features()
if dihedral_angle:
    nTotalEdgeFeatures += dihedral_angle.n_features()

Now, we define our model

In [26]:
model = graph_potential(
    n_gc_layers=nGraphConvolutionLayers,
    n_fc_layers=nFullyConnectedLayers,
    n_node_features=nTotalNodeFeatures,
    n_edge_features=nTotalEdgeFeatures,
    n_neurons=15)

Escribimos cosas con logtext (DUDA AQUÍ)

In [27]:
log_text = "\n# Model Description  \n"
log_text += "- nGraphConvolutionLayers: " + repr(nGraphConvolutionLayers) + "  \n"
log_text += "- nFullyConnectedLayers: " + repr(nFullyConnectedLayers) + "  \n"
if useCovalentRadii:
    log_text += "- Using Covalent Radii to find neighbours \n"
else:
    log_text += "- nMaxNeighbours: " + repr(nMaxNeighbours) + "  \n"
log_text += "- nTotalNodeFeatures: " + repr(nTotalNodeFeatures) + "  \n"
log_text += "- physical nodeFeatures: " + repr(nodeFeatures) + "  \n"
log_text += "- nEdgeFeatures: " + repr(nTotalEdgeFeatures) + "  \n"
log_text += "- Edge r_min: " + repr(edge_parameters["x_min"]) + " Angstrom  \n"
log_text += "- Edge r_max: " + repr(edge_parameters["x_max"]) + " Angstrom  \n"
log_text += "- Edge nFeatures: " + repr(edge_parameters["n_features"]) + " \n"
log_text += "- Edge sigma: " + repr(edge_parameters["sigma"]) + " Angstrom  \n"
log_text += "- Bond-Angle min: " + repr(bond_angle_parameters["x_min"]) + " radians  \n"
log_text += "- Bond-Angle max: " + repr(bond_angle_parameters["x_max"]) + " radians  \n"
log_text += "- Bond-Angle nFeatures: " + repr(bond_angle_parameters["n_features"]) + " \n"
log_text += "- Bond-Angle sigma: " + repr(bond_angle_parameters["sigma"]) + " radians  \n"
log_text += "- Bond-Angle normalised: " + repr(bond_angle_parameters["norm"]) + "   \n"
if dihedral_angle:
    log_text += "- Dihedral-Angle min: " + repr(dihedral_angle_parameters["x_min"]) + " radians  \n"
    log_text += "- Dihedral-Angle max: " + repr(dihedral_angle_parameters["x_max"]) + " radians  \n"
    log_text += "- Dihedral-Angle nFeatures: " + repr(dihedral_angle_parameters["n_features"]) + " \n"
    log_text += "- Dihedral-Angle sigma: " + repr(dihedral_angle_parameters["sigma"]) + " radians  \n"
    log_text += "- Dihedral-Angle normalised: " + repr(dihedral_angle_parameters["norm"]) + "   \n"

nParameters = count_model_parameters(model)
log_text += "- This model contains a total of: " + repr(nParameters) + " adjustable parameters  \n"

nEpochs = input_data.get("nEpochs", 100)
nBatch = input_data.get("nBatch", 50)
chkptFreq = input_data.get("nCheckpoint", 10)
learningRate = input_data.get("learningRate", 1.0e-3)
seed = input_data.get("randomSeed", 42)
nTrainMaxEntries = input_data.get("nTrainMaxEntries", None)
nValMaxEntries = input_data.get("nValMaxEntries", None)

log_text += "# Relaxation process  \n"
log_text += "- nEpochs: " + repr(nEpochs) + "  \n"
log_text += "- nBatch: " + repr(nBatch) + "  \n"
log_text += "- Checkpointing model and optimizer every " + repr(chkptFreq) + " epochs  \n"
log_text += "- learningRate: " + repr(learningRate) + "  \n"
log_text += "- random seed: " + repr(seed) + "  \n"

graphType = input_data.get("graphType", "covalent")
log_text += "- graph construction style: " + graphType + "  \n"

if nTrainMaxEntries:
    log_text += "- Size of training database: " + repr(nTrainMaxEntries) + "  \n"
else:
    log_text += "- Using full training database \n"
if nValMaxEntries:
    log_text += "- Size of validation/test database: " + repr(nValMaxEntries) + "  \n"
else:
    log_text += "- Using full validation/test database  \n"

Se pueden incluir fuerzas o no, de hecho no está implementado aunque igual estaría bien ponerlo cómo posible evolución del programa para poder predecir dinámica molecular por ejemplo.

Lo de loadModel sí está implementado y sirve para cargar un modelo si nos hemos quedado a mitad de simulación o similar

In [28]:
calculateForces = input_data.get("calculateForces", False)
loadModel = input_data.get("loadModel", False)

log_text += "- Using forces in fitting: " + repr(calculateForces) + "  \n"

if loadModel:
    loadModelFileName = input_data["loadModelFileName"]
    log_text += "- Starting from previous model: " + loadModelFileName + "  \n"
else:
    log_text += "- Initialising model parameters from scratch   \n"

In [29]:
descriptionText = input_data.get("descriptionText", " ")

descriptionText += log_text

We introduce the path to the data files

In [30]:
trainDir = input_data.get("trainDir", "/content/drive/MyDrive/Colab Notebooks/MoleculeDB/training_set")
valDir = input_data.get("valDir", "/content/drive/MyDrive/Colab Notebooks/MoleculeDB/validation_set")
testDir = input_data.get("testDir", "/content/drive/MyDrive/Colab Notebooks/MoleculeDB/test_set")
directories = [trainDir, valDir, testDir]

In [31]:
transformData = input_data.get("transformData", False)
# transform = SetUpDataTransform( transformData, directories )
transform = None

if transform:
    log_text += "- Using data transformation " + transformData + "   \n"

At this point, we obtain the corresponding graphs from each molecule file

In [32]:
Graphs = set_up_molecular_graphs(
    graphType,
    edge_features=edges,
    bond_angle_features=bond_angle,
    dihedral_features=dihedral_angle,
    node_feature_list=nodeFeatures,
    n_total_node_features=nTotalNodeFeatures)

Now, we prepare the graph dataset

In [33]:
trainDataset = MoleculeGraphDataSet(
    trainDir, Graphs, nMaxEntries=nTrainMaxEntries, seed=seed, transform=transform
)

if nTrainMaxEntries:
    nTrain = nTrainMaxEntries
else:
    nTrain = len(trainDataset)

valDataset = MoleculeGraphDataSet(
    valDir, Graphs, nMaxEntries=nValMaxEntries, seed=seed, transform=transform
)

if nValMaxEntries:
    nValidation = nValMaxEntries
else:
    nValidation = len(valDataset)

testDataset = MoleculeGraphDataSet(
    testDir, Graphs, nMaxEntries=nValMaxEntries, seed=seed, transform=transform
)

trainLoader = DataLoader(trainDataset, batch_size=nBatch, num_workers=0)
valLoader = DataLoader(valDataset, batch_size=nBatch, num_workers=0)

We define a tensorboard writer to monitor the fitting process

In [34]:
timeString = datetime.now().strftime("%d%m%Y-%H%M%S")
fileName = "5dihedralfeat-" + repr(nMaxNeighbours) + "nn-" + repr(nGraphConvolutionLayers) + "gcl-" + repr(nFullyConnectedLayers) + "fcl"
logFile = "/content/drive/MyDrive/Colab Notebooks/MoleculeDB/runs/" + fileName + "-" + timeString

saveModelFileName = (
    "GraphPotential-"
    + repr(nMaxNeighbours)
    + "nn-"
    + repr(nGraphConvolutionLayers)
    + "gcl-"
    + repr(nFullyConnectedLayers)
    + "fcl-"
    + timeString
    + ".tar"
)

writer = SummaryWriter(logFile)

description = markdown.markdown(descriptionText)

writer.add_text("Description", description)

Optimizier

In [35]:
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)

# if we are to use a pre-saved model and optimizer, load their parameters here

if loadModel:

    # model = torch.load( loadModelFileName )

    checkpoint = torch.load(loadModelFileName)

    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    n_start = checkpoint["epoch"]

else:

    n_start = 0

lossFunction = torch.nn.MSELoss()

Now, we set-up the callbacks, if any

In [36]:
anyCallBacks = input_data.get("callbacks", None)

callbacks = set_up_callbacks(anyCallBacks, optimizer)

Training process

In [ ]:
print("epoch      train-loss      validation-loss")
print("------------------------------------------")

fit_model(
    nEpochs,
    model,
    lossFunction,
    optimizer,
    nTrain,
    nValidation,
    trainLoader,
    valLoader,
    n_epoch_0=n_start,
    calculate_forces=calculateForces,
    weight=1.0e0,
    writer=writer,
    callbacks=callbacks,
    check_point_path=saveModelFileName,
    check_point_freq=chkptFreq)

epoch      train-loss      validation-loss
------------------------------------------
0,61859.25128746033,1594.6965265274048
1,1436.6725668907166,1584.2344427108765
2,1419.558138847351,1569.3140506744385
3,1394.1006126403809,1552.2652769088745
4,1357.2668256759644,1520.2881956100464
5,1287.7611966133118,1418.2551050186157
6,1080.4236857891083,1045.6035375595093
7,780.2718715667725,746.5925002098083
8,573.1636034250259,580.8906102180481
9,452.05492329597473,468.6547780036926
10,342.3220602273941,344.44878339767456
11,276.01938033103943,295.0667953491211
12,238.80330574512482,265.9241724014282
13,211.34364545345306,238.8084065914154
14,188.51867192983627,222.65822887420654
15,169.35104590654373,220.48656284809113
16,153.83321249485016,219.92170453071594
17,141.99475413560867,213.9182686805725
18,131.19841983914375,193.77163469791412
19,118.4835532605648,151.18695676326752
20,102.13494443893433,114.67133969068527
21,88.10101741552353,100.96525728702545
22,76.55519454181194,96.748240590095

Let us build a histogram of the errors

In [ ]:
testLoader = DataLoader(testDataset, batch_size=1, num_workers=0)

print("         TEST SAMPLE ENERGIES             ")
print("------------------------------------------")

e = []
p = []

with torch.no_grad():

    file = open(logFile + "prediction.csv", "w")
    file.write("exact, predicted" + os.linesep)

    for sample in testLoader:
        
        energy = model(sample.x, sample.edge_index, sample.edge_attr, sample.batch)
        e.append(sample.y.item())
        p.append(energy.item())
        txt = repr(energy.item()) + ", " + repr(sample.y.item())

        file.write(txt + os.linesep)
        #print(txt)

    file.close()
prediction = np.array(p)
exact = np.array(e)
error = prediction - exact

if writer is not None:

    eMax = np.max(exact)
    eMin = np.min(exact)

    x = np.linspace(eMin, eMax, 100)

    fig, ax = plt.subplots()
    #figPredVsExN = plt.figure()

    ax.scatter(exact, prediction, color = '#0B00A8', label="Model predictions")
    ax.plot(x, x, color = 'red', label="Exact")

    ax.set_ylabel(r'Predicted energies', fontdict = {'fontsize':26, 'color':'k'})
    ax.set_xlabel(r'QM9 theoretical energies', fontdict = {'fontsize':24, 'color':'k'})
    
    ax.legend()

    plt.show()

    writer.add_figure("Prediction vs. exact ", fig, nEpochs)
    writer.add_histogram("Distribution of errors normalised data (prediction - exact)", error)

writer.close()